In [1]:
# Load libraries
import pandas as pd

In [2]:
# Load datasets
df_routes = pd.read_csv('../data/df_routes_opt.csv')
df_routes.head()

,ori,des,osrm_dist,route_geometry
0,Estación del Nord,Falla Gayano Lluch,714.8,"[[-0.377108, 39.467366], [-0.379316, 39.488637..."
1,Falla Gayano Lluch,Falla Monestir de Poblet – Aparicio Albiñana (...,2031.6,"[[-0.382982, 39.492747], [-0.383104, 39.493083..."
2,Falla Monestir de Poblet – Aparicio Albiñana (...,Falla Quart-Extramurs-Velázquez,2237.1,"[[-0.397088, 39.483926], [-0.396998, 39.483797..."
3,Falla Quart-Extramurs-Velázquez,Falla Na Jordana,2757.2,"[[-0.407345, 39.472864], [-0.407455, 39.472843..."
4,Falla Na Jordana,Falla Reina-Paz-San Vicente (Falla Tio Pep),1193.6,"[[-0.38001, 39.480319], [-0.379891, 39.479779]..."


In [3]:
df_fallas = pd.read_csv('../data/fallas.csv')
# Add start point
# Crear el nuevo registro
ayuntamiento = pd.DataFrame([{
    "Falla": "Estación del Nord",
    "Enlace": "https://www.google.com/maps/place/Ayuntamiento+de+Valencia/@39.4698068,-0.3771595,58m/data=!3m2!1e3!5s0xd604f4b7f4c477b:0x5596cd1cc0ee5fb8!4m6!3m5!1s0xd6048ad7bf45aab:0x2ba4baea5a36e1e2!8m2!3d39.4697956!4d-0.3770532!16s%2Fg%2F11cnp7lthl?entry=ttu&g_ep=EgoyMDI1MDMxMi4wIKXMDSoJLDEwMjExNDUzSAFQAw%3D%3D",
    "lat": 39.4674694952255,
    "lon": -0.3772975952898449
}])
df_fallas = pd.concat([ayuntamiento, df_fallas], ignore_index=True)
df_fallas

,Falla,Enlace,lat,lon
0,Estación del Nord,https://www.google.com/maps/place/Ayuntamiento...,39.467469,-0.377298
1,Falla Convento Jerusalén-Matemático Marzal,https://www.google.com/maps/place/Falla+Conven...,39.466441,-0.379912
2,Falla Cuba-Literato Azorín,https://www.google.com/maps/place/Falla+Cuba+L...,39.459464,-0.376410
3,Falla Sueca-Literato Azorín,https://www.google.com/maps/place/Falla+Sueca+...,39.460148,-0.376865
4,Falla Almirante Cadarso-Conde de Altea,https://www.google.com/maps/place/Falla+Almira...,39.465822,-0.368480
5,Falla Na Jordana,https://www.google.com/maps/place/Falla+Na+Jor...,39.480291,-0.380225
6,Falla Plaza del Pilar,https://www.google.com/maps/place/Falla+Plaza+...,39.471361,-0.383138
7,Falla Regne de Valencia-Duque de Calabria,https://www.google.com/maps/place/A.C.+Falla+R...,39.463688,-0.366951
8,Falla Exposición-Misser Mascó,https://www.google.com/maps/place/Falla+Exposi...,39.475290,-0.361864
9,Falla Monestir de Poblet – Aparicio Albiñana (...,https://www.google.com/maps/place/Falla+Monest...,39.483891,-0.397193


In [4]:
import geopandas as gpd
import pandas as pd
import ast
from shapely.geometry import LineString

# Clean and convert the route_geometry column into a Python list (coordinates)
def create_line_string(geometry_str):
    try:
        # Remove any unwanted characters or spaces before parsing
        geometry_str_clean = geometry_str.strip()
        
        # Check if the string starts and ends with square brackets (list format)
        if geometry_str_clean.startswith('[') and geometry_str_clean.endswith(']'):
            # Safely convert string to list of coordinates
            coords = ast.literal_eval(geometry_str_clean)
            # Create a LineString geometry from the coordinates
            return LineString(coords)
        else:
            raise ValueError(f"Invalid geometry format: {geometry_str}")
    except (ValueError, SyntaxError, TypeError) as e:
        print(f"Error processing geometry: {e}")
        return None  # Return None for invalid geometries

def df_to_gdf(df, route_geometry_col='route_geometry'):


    # Apply the function to create the 'geometry' column containing LineString geometries
    df['geometry'] = df['route_geometry'].apply(create_line_string)

    # Now convert the DataFrame to a GeoDataFrame
    gdf_routes = gpd.GeoDataFrame(df, geometry='geometry')

    # Set the coordinate reference system (CRS) if known (for example, EPSG:4326 for WGS84)
    gdf_routes.set_crs('EPSG:4326', allow_override=True, inplace=True)

    return gdf_routes.drop(columns=[route_geometry_col])

# Check the GeoDataFrame
gdf_routes = df_to_gdf(df_routes)
gdf_routes.head()

,ori,des,osrm_dist,geometry
0,Estación del Nord,Falla Gayano Lluch,714.8,"LINESTRING (-0.37711 39.46737, -0.37932 39.488..."
1,Falla Gayano Lluch,Falla Monestir de Poblet – Aparicio Albiñana (...,2031.6,"LINESTRING (-0.38298 39.49275, -0.3831 39.4930..."
2,Falla Monestir de Poblet – Aparicio Albiñana (...,Falla Quart-Extramurs-Velázquez,2237.1,"LINESTRING (-0.39709 39.48393, -0.397 39.4838,..."
3,Falla Quart-Extramurs-Velázquez,Falla Na Jordana,2757.2,"LINESTRING (-0.40734 39.47286, -0.40746 39.472..."
4,Falla Na Jordana,Falla Reina-Paz-San Vicente (Falla Tio Pep),1193.6,"LINESTRING (-0.38001 39.48032, -0.37989 39.479..."


In [5]:
df_fallas

,Falla,Enlace,lat,lon
0,Estación del Nord,https://www.google.com/maps/place/Ayuntamiento...,39.467469,-0.377298
1,Falla Convento Jerusalén-Matemático Marzal,https://www.google.com/maps/place/Falla+Conven...,39.466441,-0.379912
2,Falla Cuba-Literato Azorín,https://www.google.com/maps/place/Falla+Cuba+L...,39.459464,-0.376410
3,Falla Sueca-Literato Azorín,https://www.google.com/maps/place/Falla+Sueca+...,39.460148,-0.376865
4,Falla Almirante Cadarso-Conde de Altea,https://www.google.com/maps/place/Falla+Almira...,39.465822,-0.368480
5,Falla Na Jordana,https://www.google.com/maps/place/Falla+Na+Jor...,39.480291,-0.380225
6,Falla Plaza del Pilar,https://www.google.com/maps/place/Falla+Plaza+...,39.471361,-0.383138
7,Falla Regne de Valencia-Duque de Calabria,https://www.google.com/maps/place/A.C.+Falla+R...,39.463688,-0.366951
8,Falla Exposición-Misser Mascó,https://www.google.com/maps/place/Falla+Exposi...,39.475290,-0.361864
9,Falla Monestir de Poblet – Aparicio Albiñana (...,https://www.google.com/maps/place/Falla+Monest...,39.483891,-0.397193


# Sort the fallas coordinates

In [6]:
df_fallas = pd.merge(df_fallas, gdf_routes["ori"], left_on='Falla',right_on="ori", how="right")
df_fallas.reset_index(drop=True, inplace=True)
df_fallas.head()

,Falla,Enlace,lat,lon,ori
0,Estación del Nord,https://www.google.com/maps/place/Ayuntamiento...,39.467469,-0.377298,Estación del Nord
1,Falla Gayano Lluch,https://www.google.com/maps/place/Falla+Gayano...,39.492748,-0.382976,Falla Gayano Lluch
2,Falla Monestir de Poblet – Aparicio Albiñana (...,https://www.google.com/maps/place/Falla+Monest...,39.483891,-0.397193,Falla Monestir de Poblet – Aparicio Albiñana (...
3,Falla Quart-Extramurs-Velázquez,https://www.google.com/maps/place/Associaci%C3...,39.472837,-0.407336,Falla Quart-Extramurs-Velázquez
4,Falla Na Jordana,https://www.google.com/maps/place/Falla+Na+Jor...,39.480291,-0.380225,Falla Na Jordana


# Create the map

In [7]:
import folium
import folium
from folium import Icon

def create_route_map(df_coordinates, df_routes):
    # Crear el mapa centrado en la primera ruta
    m = folium.Map(location=[df_coordinates['lat'].iloc[0], df_coordinates['lon'].iloc[0]], zoom_start=12.5)

    # Agregar puntos con un ícono de fuego y el texto con el índice
    for i, row in df_coordinates.iterrows():
        # Crear el texto con el índice
        popup_text = f'Orden: {i} - Falla: {row["Falla"]}'

        # Primer punto con un ícono de fuego de color azul
        if i == 0:
            folium.Marker(
                location=[row['lat'], row['lon']],
                icon=folium.Icon(icon="fire", prefix="fa", icon_color='blue'),
                popup=popup_text,
                tooltip=popup_text
            ).add_to(m)
        else:
            folium.Marker(
                location=[row['lat'], row['lon']],
                icon=folium.Icon(icon="fire", prefix="fa", icon_color='red'),
                popup=popup_text,
                tooltip=popup_text
            ).add_to(m)

    # Iterar sobre el GeoDataFrame y agregar cada ruta como PolyLine
    for idx, row in df_routes.iterrows():
        # Verificar si la geometría es un LineString válido
        if row['geometry'].geom_type == 'LineString':
            # Extraer las coordenadas de la geometría LineString y convertirla a una lista de tuplas
            coordinates = list(row['geometry'].coords)  # Lista de tuplas (lon, lat)
            
            # Agregar una PolyLine al mapa
            folium.PolyLine(locations=[[co[1], co[0]] for co in coordinates], color="blue", weight=2.5, opacity=1).add_to(m)

    return m

# Usar la función para crear el mapa con los datos de coordenadas y rutas
create_route_map(df_fallas, df_routes)


In [8]:
import folium
import matplotlib.pyplot as plt
import numpy as np
from folium import Icon

def generate_color_palette(num_colors):
    """Genera una lista de colores de manera gradual"""
    colors = plt.cm.viridis(np.linspace(0, 1, num_colors))
    # Convertir los colores de rgba a hex
    return ['#%02x%02x%02x' % (int(r*255), int(g*255), int(b*255)) for r, g, b, a in colors]

def create_route_map(df_coordinates, df_routes):
    # Crear el mapa centrado en la primera ruta
    m = folium.Map(tiles='cartodbpositron', location=[df_coordinates['lat'].iloc[0], df_coordinates['lon'].iloc[0]], zoom_start=12.5)

    # Agregar puntos con un ícono de fuego y el texto con el índice
    for i, row in df_coordinates.iterrows():
        popup_text = f'Orden: {i} - Falla: {row["Falla"]}'

        # Primer punto con un ícono de fuego de color azul
        if i == 0:
            folium.Marker(
                location=[row['lat'], row['lon']],
                icon=folium.Icon(icon="fire", prefix="fa", icon_color='blue'),
                popup=popup_text,
                tooltip=popup_text
            ).add_to(m)
        else:
            folium.Marker(
                location=[row['lat'], row['lon']],
                icon=folium.Icon(icon="fire", prefix="fa", icon_color='red'),
                popup=popup_text,
                tooltip=popup_text
            ).add_to(m)

    # Generar la paleta de colores para las rutas
    num_routes = len(df_routes)
    color_palette = generate_color_palette(num_routes)

    # Iterar sobre el GeoDataFrame y agregar cada ruta como PolyLine
    for idx, row in df_routes.iterrows():
        if row['geometry'].geom_type == 'LineString':
            coordinates = list(row['geometry'].coords)  # Lista de tuplas (lon, lat)
            
            # Asignar un color de la paleta a esta ruta
            route_color = color_palette[idx]
            
            # Agregar una PolyLine al mapa con color gradual
            folium.PolyLine(
                locations=[[co[1], co[0]] for co in coordinates],
                color=route_color,
                weight=2.5,
                opacity=1
            ).add_to(m)

    return m

# Usar la función para crear el mapa con los datos de coordenadas y rutas
create_route_map(df_fallas, df_routes)